(model_configuration)=
# Model Configuration

This notebook provides details about the model configuration in PyMC-Marketing. This API provides a very flexible way to express the model Priors and likelihoods so that we can have enough expressiveness to build custom models.

## Setup

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

from pymc_marketing.model_config import ModelConfigError, parse_model_config
from pymc_marketing.prior import Prior

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
seed = sum(map(ord, "Create priors to reflect your marketing beliefs"))
rng = np.random.default_rng(seed)

## Prior Distributions

The {class}`~pymc_marketing.prior.Prior` class is our way to expression distributions and relationships between them.

### Basic Usage

Every prior will need a distribution name that comes from `PyMC`. View the full list in the documentation [here](https://www.pymc.io/projects/docs/en/stable/api/distributions.html).

In [ ]:
scalar_distribution = Prior("Normal")
scalar_distribution

If not, then an exception will be raised.

In [ ]:
try:
    Prior("UnknownDistribution")
except Exception as e:
    print(e)

Specific parameters can be passed as **keyword arguments** but are not required if the `PyMC` distribution has defaults.

In [ ]:
scalar_distribution = Prior("Normal", mu=0, sigma=3.5)
scalar_distribution

There will be a check at initialization against the `PyMC` distribution.

In [ ]:
try:
    Prior("Normal", mu=0, b=1)
except Exception as e:
    print(e)

However, there are some limitations to that. Take this invalid parameterization below:

In [ ]:
invalid_distribution = Prior("Normal", mu=1, sigma=1, tau=2)

The `create_variable` method is used to make variables from this distribution and it isn't until then that there is an error.

In [ ]:
with pm.Model():
    try:
        invalid_distribution.create_variable("mu")
    except Exception as e:
        print(e)

This goes for incorrect values as well!

In [ ]:
invalid_distribution = Prior("Normal", mu=1, sigma=-1)
invalid_distribution

The `dims` parameter are used to express the dimensions of the distribution. This variable is not part of a model so the values are not reflected with the distribution itself.

In [ ]:
vector_distribution = Prior("Normal", dims="channel")
vector_distribution

If there are dimensions, then the coords need to exist in the larger model. But they exist separately and can defined in advance.

```{tip}
Each `Prior` instance is just instructions to create a variable, so they can be used multiple times!
```

In [ ]:
coords = {"channel": ["C1", "C2"]}
with pm.Model(coords=coords) as model:
    alpha = vector_distribution.create_variable("alpha")
    beta = vector_distribution.create_variable("beta")

pm.model_to_graphviz(model)

If the coords were not specified then this would cause a PyMC error.

In [ ]:
with pm.Model() as model:
    try:
        vector_distribution.create_variable("var")
    except Exception as e:
        print(e)

The variables can get arbitrarily large as well by providing additional dims in a tuple.

In [ ]:
matrix_distribution = Prior("Normal", dims=("channel", "geo"))
matrix_distribution

In [ ]:
tensor_distribution = Prior("Normal", dims=("channel", "geo", "store"))
tensor_distribution

### Hierarchical Variables

Hierarchical variables can be defined by using distributions as the parameters of another distribution. The parent distributions will usually have a larger dimensionality than each of the parameters.

In [ ]:
hierarchical_variable = Prior(
    "Normal",
    mu=Prior("Normal"),
    sigma=Prior("HalfNormal"),
    dims="channel",
)

We can use the `to_graph` method to create visualize the variable with dummy coordinates.

```{note}
No need to worry about variable naming! The child parameters will all be automatically named based on the parent.
```

In [ ]:
hierarchical_variable.to_graph()

```{warning}
The validity of the parameter values will not be checked. 
```

There can be negative sigma parameter in a Normal distribution whether that comes from value or another distribution!

In [ ]:
Prior("Normal", mu=1, sigma=-1)

In [ ]:
Prior("Normal", sigma=Prior("Normal"))

```{tip}
Model reparamterizations can help with model convergence depending on the model posterior!
```

For Normal distribution, the common non-centered parameterization is supported with the `centered` flag.

In [ ]:
non_centered_hierarchical_variable = Prior(
    "Normal",
    mu=Prior("Normal"),
    sigma=Prior("HalfNormal"),
    dims="channel",
    # Flag for non-centered
    centered=False,
)
non_centered_hierarchical_variable.to_graph()

Other distributions can be hierarchical as well. Just use distributions for parameters of a parent distribution. For instance, the Beta distribution has two positive parameters, `alpha` and `beta` which can be reflect as HalfNormal distributions.

In [ ]:
zero_to_one_variable = Prior(
    "Beta",
    alpha=Prior("HalfNormal"),
    beta=Prior("HalfNormal"),
    dims="channel",
)
zero_to_one_variable.to_graph()

### Transformations

The `transform` variable can be used for any of the distributions to change its domain.

These transformations will be taken from `pytensor.tensor` or `pm.math` in that order.

Below is a non-centered hierarchical Normal distribution that is put through a sigmoid function in order to change the domain to (0, 1).

In [ ]:
hierarchical_zero_to_one_distribution = Prior(
    "Normal",
    mu=Prior("Normal"),
    sigma=Prior("HalfNormal"),
    dims="channel",
    centered=False,
    transform="sigmoid",
)
hierarchical_zero_to_one_distribution.to_graph()

### Prior Visualization

For scalar distributions, make use of the `preliz` attribute of the model in order to visualize. 

```{tip}
Defaults for the parameters will have to be provided or there will be an error.
```

In [ ]:
beta_distribution = Prior("Beta", alpha=1, beta=4)

beta_distribution.preliz.plot_pdf();

```{tip}
Using the `constrain` method can help hone in on a prior. It is a wrapper around [PreliZ's `maxent` function](https://preliz.readthedocs.io/en/latest/api_reference.html#preliz.unidimensional.maxent).
```

In [ ]:
constrained_distribution = Prior("Normal").constrain(lower=4, upper=6)
constrained_distribution

To keep a parameter value fixed, pass it to the Prior instance. In the following example, the mean is fixed at 2. 

In [ ]:
constrained_distribution = Prior("Gamma", mu=2).constrain(lower=1, upper=10)
constrained_distribution

For more complicated distributions, consider using the `sample_prior` method in order to access the prior. Or PreliZ's functions like [predictive explorer](https://preliz.readthedocs.io/en/latest/examples/observed_space_examples.html#predictive-explorer).


```{note}
Any coordinates will have to be passed like in `pm.Model`.
```

In [ ]:
complicated_distribution = Prior(
    "Normal",
    mu=Prior("Normal", sigma=1),
    sigma=Prior("Normal", mu=-1, sigma=0.15, transform="exp"),
    centered=False,
    dims="channel",
    transform="sigmoid",
)

coords = {
    "channel": ["C1", "C2"],
}
prior = complicated_distribution.sample_prior(
    coords=coords, samples=1_000, random_seed=rng
)
prior

The prior variables can be visualized in whatever way works best. Here is a KDE of the variables to see the marginal distributions.

In [ ]:
fig, axes = plt.subplots(
    nrows=2, ncols=1, figsize=(10, 9), sharex=True, sharey=True, layout="constrained"
)

az.plot_posterior(prior, var_names=["var"], grid=(2, 1), ax=axes)
fig.suptitle("Prior Distribution", fontsize=18, fontweight="bold");

Use any workflow to make sense of the prior distribution! For instance, the joint distribution which shows these two channels are correlated due to the hierarchical generation.

In [ ]:
def plot_correlation(df: pd.DataFrame, x: str = "C1", y: str = "C2") -> plt.Axes:
    corr = df.loc[:, [x, y]].corr().iloc[0, 1]
    title = f"Joint distribution between {x} and {y} ({corr = :.3f})"
    return df.plot.scatter(x=x, y=y, title=title)


ax = prior["var"].to_series().unstack().pipe(plot_correlation)
padding = 0.025
bounds = (0 - padding, 1 + padding)
ax.set(xlim=bounds, ylim=bounds);

### Automatic Broadcasting

The broadcasting of the dims will be handled automatically.

For instance, the `mu` of the variable needs to be transposed in order to work with `("channel", "geo")` dims.

With all this functionality, we can see that the prior distributions can become quite expressive!

In [ ]:
def create_2d_variable(mu_dims, sigma_dims) -> Prior:
    mu = Prior(
        "Normal",
        mu=Prior("Normal"),
        sigma=Prior("HalfNormal"),
        dims=mu_dims,
    )
    sigma = Prior(
        "Normal",
        mu=Prior("Normal"),
        sigma=Prior("HalfNormal"),
        centered=False,
        dims=sigma_dims,
        transform="exp",
    )
    return Prior(
        "Normal",
        mu=mu,
        sigma=sigma,
        dims=("channel", "geo"),
    )


variable_2d = create_2d_variable(mu_dims="channel", sigma_dims="geo")
variable_2d.to_graph()

And the user can spend more time thinking about just the assumptions of the variables rather than logic to ensure the dimensions work.

In [ ]:
different_assumptions_2d = create_2d_variable(mu_dims="channel", sigma_dims="channel")

different_assumptions_2d.to_graph()

### Serialization

The `to_json` and `from_json` methods can be helpful for storage of the distributions.

In [ ]:
variable_2d_dict = variable_2d.to_json()
variable_2d_dict

In [ ]:
Prior.from_json(variable_2d_dict).to_graph()

## Use in PyMC-Marketing

Distributions will be expressed in this manner throughout the package including but not limited to: 

- MMM components
    - [media transformations](https://www.pymc-marketing.io/en/stable/notebooks/mmm/mmm_components.html#media-transformations)
 
### Backwards compatibility

The `from_json` method will create a `Prior` instance from the previous format.

For instance, take this previous configuration: 

In [ ]:
old_model_config = {
    "alpha": {
        "dist": "Normal",
        "kwargs": {
            "mu": 0,
            "sigma": 1,
        },
    },
    "beta": {
        "dist": "Laplace",
        "kwargs": {
            "mu": 1,
            "b": 0.5,
        },
    },
}

This can be parsed with the `Prior.from_json` constructor for each key. Much more consise too!

In [ ]:
new_model_config = {
    name: Prior.from_json(key) for name, key in old_model_config.items()
}

new_model_config

In [ ]:
new_model_config["alpha"].to_json()

The `parse_model_config` function will do just this and is used internally. It also provides some deprecation warnings.

In [ ]:
parse_model_config(old_model_config)

As well as the ability to catch some errors while parsing.

In [ ]:
invalid_model_config = {
    "alpha": {
        "dist": "InvalidDistribution",
    },
    "beta": {
        "dist": "Normal",
        "kwargs": {"mu": "one", "sigma": "two"},
    },
    "gamma": {
        "dist": "HalfNormal",
        "kwargs": {"mu": 1},
    },
}

try:
    parse_model_config(invalid_model_config)
except ModelConfigError as e:
    print(e)

## Appendix: Validation

Behind the scenes, the `Prior` class uses th `validate_call` from [`Pydantic`](https://docs.pydantic.dev/latest/concepts/validators/) to ensure that the parameters are valid.`

In [ ]:
try:
    Prior()
except Exception as e:
    print(e)

In [ ]:
try:
    Prior(1)
except Exception as e:
    print(e)

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p pymc_marketing,pytensor